In [10]:
local_model = "/Users/maxhuang/projects/llm/models/Qwen/Qwen2.5-Coder-14B-Instruct-GGUF/qwen2.5-coder-14b-instruct-q4_0.gguf"
import multiprocessing

from langchain_community.chat_models import ChatLlamaCpp

llm = ChatLlamaCpp(
    temperature=0,
    model_path=local_model,
    n_ctx=10000,
    n_gpu_layers=8,
    n_batch=300,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    max_tokens=512,
    n_threads=multiprocessing.cpu_count() - 1,
    repeat_penalty=1.5,
    top_p=0.5,
    verbose=False,
)

In [3]:
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain.prompts import PromptTemplate
import sqlite3


In [12]:
def get_schema_details(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    schema_info = {}
    for table in tables:
        table_name = table[0]
        # Get columns
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        # Get foreign keys
        cursor.execute(f"PRAGMA foreign_key_list({table_name})")
        foreign_keys = cursor.fetchall()
        
        schema_info[table_name] = {
            'columns': [(col[1], col[2]) for col in columns],  # name, type
            'foreign_keys': [(fk[3], fk[2], fk[4]) for fk in foreign_keys]  # table, from, to
        }
    
    conn.close()
    #print(schema_info)
    return schema_info

def create_smart_prompt(schema_info):
    schema_text = "Database Schema:\n"
    for table, info in schema_info.items():
        schema_text += f"\nTable: {table}\n"
        schema_text += "Columns:\n"
        for col_name, col_type in info['columns']:
            schema_text += f"- {col_name} ({col_type})\n"
        if info['foreign_keys']:
            schema_text += "Relations:\n"
            for ref_table, from_col, to_col in info['foreign_keys']:
                schema_text += f"- Links to {ref_table} ({from_col} -> {to_col})\n"
    #print(schema_text)
    
    return PromptTemplate(
        template=f"""{schema_text}
        
        Given this schema, write a SQL query to answer: {{question}}
        Consider table relationships and use appropriate JOINs.
        SQL Query:""",
        input_variables=["question"]
    )


In [8]:
print(get_schema_details("chinook.db"))

{'albums': {'columns': [('AlbumId', 'INTEGER'), ('Title', 'NVARCHAR(160)'), ('ArtistId', 'INTEGER')], 'foreign_keys': [('ArtistId', 'artists', 'ArtistId')]}, 'sqlite_sequence': {'columns': [('name', ''), ('seq', '')], 'foreign_keys': []}, 'artists': {'columns': [('ArtistId', 'INTEGER'), ('Name', 'NVARCHAR(120)')], 'foreign_keys': []}, 'customers': {'columns': [('CustomerId', 'INTEGER'), ('FirstName', 'NVARCHAR(40)'), ('LastName', 'NVARCHAR(20)'), ('Company', 'NVARCHAR(80)'), ('Address', 'NVARCHAR(70)'), ('City', 'NVARCHAR(40)'), ('State', 'NVARCHAR(40)'), ('Country', 'NVARCHAR(40)'), ('PostalCode', 'NVARCHAR(10)'), ('Phone', 'NVARCHAR(24)'), ('Fax', 'NVARCHAR(24)'), ('Email', 'NVARCHAR(60)'), ('SupportRepId', 'INTEGER')], 'foreign_keys': [('SupportRepId', 'employees', 'EmployeeId')]}, 'employees': {'columns': [('EmployeeId', 'INTEGER'), ('LastName', 'NVARCHAR(20)'), ('FirstName', 'NVARCHAR(20)'), ('Title', 'NVARCHAR(30)'), ('ReportsTo', 'INTEGER'), ('BirthDate', 'DATETIME'), ('HireDate

In [13]:
# Initialize database and chain
db_path = "chinook.db"  # Update with your DB path
schema_info = get_schema_details(db_path)
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# Create chain with schema-aware prompt
smart_chain = SQLDatabaseChain.from_llm(
    llm=llm,  # Your existing LLM instance
    db=db,
    prompt=create_smart_prompt(schema_info),
    verbose=True
)



> Entering new SQLDatabaseChain chain...
Error: Missing some input keys: {'query'}


> Entering new SQLDatabaseChain chain...
Error: Missing some input keys: {'query'}


> Entering new SQLDatabaseChain chain...
Error: Missing some input keys: {'query'}


> Entering new SQLDatabaseChain chain...
Error: Missing some input keys: {'query'}


In [14]:
# Example query
def query_database(question: str):
    try:
        return smart_chain.run(question)
    except ValueError as e:
        print(f"Error: {e}")
        return None
    
query_database("How many employees are there?")
query_database("What is the name of the first employee?")
query_database("Which customer has the most invoices?")
query_database("List all music genres in the database")



> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:Error: Missing some input keys: {'question'}


> Entering new SQLDatabaseChain chain...
What is the name of the first employee?
SQLQuery:Error: Missing some input keys: {'question'}


> Entering new SQLDatabaseChain chain...
Which customer has the most invoices?
SQLQuery:Error: Missing some input keys: {'question'}


> Entering new SQLDatabaseChain chain...
List all music genres in the database
SQLQuery:Error: Missing some input keys: {'question'}


/var/folders/x2/zk253tmn003dnbcnhrtjggc80000gn/T/ipykernel_69627/1457577392.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return smart_chain.run(question)
